# Joining rows in very, very, very large lists of categories
---

## Import the necessary packages

In [ ]:
import pandas as pd                        # Pandas to load the data initially
import numpy as np                         # Mathematical operations package, allowing also for missing values representation
import torch                               # PyTorch for tensor and deep learning operations
import data_utils as du                    # Data science and machine learning relevant methods

In [ ]:
du.set_pandas_library('pandas')

Allow pandas to show more columns:

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [ ]:
import pixiedust                           # Debugging in Jupyter Notebook cells

## Initialize variables

Data that we'll be using:

In [ ]:
id_col = np.concatenate([np.repeat(1, 25), 
                         np.repeat(2, 17), 
                         np.repeat(3, 56), 
                         np.repeat(4, 138), 
                         np.repeat(5, 2000), 
                         np.repeat(6, 100000)])
id_col

In [ ]:
ts_col = np.concatenate([np.repeat(0, 25), 
                         np.repeat(0, 17), 
                         np.repeat(0, 16), np.repeat(1, 40), 
                         np.repeat(0, 30), np.repeat(1, 8), np.repeat(2, 50), np.repeat(3, 50), 
                         np.repeat(0, 1000), np.repeat(1, 1000),
                         np.repeat(0, 100000)])
ts_col

In [ ]:
categ_col = np.concatenate([np.random.randint(0, 50, size=(102200)), np.repeat(np.nan, 36)])
np.random.shuffle(categ_col)
categ_col

In [ ]:
data = np.column_stack([id_col, ts_col, categ_col])
data

In [ ]:
data_df = pd.DataFrame(data, columns=['id', 'ts', 'categ_var'])
data_df

In [ ]:
# data_df.head(1000)

In [ ]:
data_df.info(memory_usage='deep')

## Enumerate categories

In [ ]:
# data_df.categ_var, enum_dict = du.embedding.enum_categorical_feature(data_df, 'categ_var', nan_value=0, forbidden_digit=0)
# data_df

In [ ]:
# enum_dict

In [ ]:
%%time
pd.Categorical(data_df['categ_var'])

In [ ]:
%%time
x1 = pd.get_dummies(data_df, columns=['categ_var'])
x1.head()

In [ ]:
x1.dtypes

In [ ]:
%%time
x2 = pd.get_dummies(data_df, columns=['categ_var'], sparse=True)
x2.head()

In [ ]:
x2.dtypes

In [ ]:
%%time
x3, _ = du.data_processing.one_hot_encoding_dataframe(data_df, columns='categ_var', join_rows=False, 
                                                      get_new_column_names=True, inplace=False)
x3.head()

In [ ]:
x4.dtypes

In [ ]:
%%time
x4, _ = du.data_processing.one_hot_encoding_dataframe(data_df, columns='categ_var', join_rows=False, 
                                                      get_new_column_names=True, inplace=True)
x4.head()

In [ ]:
x5.dtypes

In [ ]:
data_df, ohe_columns = du.data_processing.one_hot_encoding_dataframe(data_df, columns='categ_var', join_rows=False, 
                                                                     get_new_column_names=True, inplace=True)
data_df

In [ ]:
ohe_columns

In [ ]:
data_df.info(memory_usage='deep')

In [ ]:
data_df[ohe_columns] = data_df[ohe_columns].astype('boolean')

In [ ]:
data_df.info(memory_usage='deep')

## Join rows

### Use all values

In [ ]:
data_df = du.embedding.join_repeated_rows(data_df, cat_feat='categ_var', id_columns=['id', 'ts'], unique=False, inplace=True)
data_df

#### Convert to a PyTorch tensor

In [ ]:
data_tnsr = torch.from_numpy(data_df.values)
data_tnsr

In [ ]:
torch.max(data_tnsr)

#### Save and load the dataframe again

In [ ]:
data_df.to_csv('random_big_data.csv')

In [ ]:
data_df.read_csv('random_big_data.csv')
data_df = data_df.drop(columns='Unnamed: 0')
data_df

In [ ]:
data_df.info(memory_usage='deep')

### Only use unique category occurences

In [ ]:
# data_df_set = data_df.groupby(['id', 'ts']).categ_var.apply(lambda df: set(df)).reset_index()
# data_df_set

In [ ]:
# data_df_set.info(memory_usage='deep')

In [ ]:
data_df = du.embedding.join_repeated_rows(data_df, cat_feat='categ_var', id_columns=['id', 'ts'], unique=True, inplace=True)
data_df

In [ ]:
data_df.info(memory_usage='deep')

**Comment:** It would be much more pleasant to the eye (and easier to handle in code) if the categorical column was just converted to a series of `set` objects. However, as we can see from the info outputs, using sets instead of large int values can make the dataframe use much more memory (depends on the data, but it can even use 200 times more memory).

In [ ]:
data_df.categ_var.max()

In [ ]:
du.utils.get_full_number_string(data_df.categ_var.astype('float64').max())

In [ ]:
int(data_df.categ_var.astype('float64')[0])

#### Convert to a PyTorch tensor

In [ ]:
data_df.iloc[10].values

In [ ]:
data_df.categ_var.values

In [ ]:
data_tnsr = torch.from_numpy(data_df.values)
data_tnsr

In [ ]:
torch.max(data_tnsr)

#### Save and load the dataframe again

In [ ]:
data_df.to_csv('random_big_data.csv')

In [ ]:
data_df = pd.read_csv('random_big_data.csv')
data_df = data_df.drop(columns='Unnamed: 0')
data_df

In [ ]:
data_df.info(memory_usage='deep')

**Comments:**
* It looks like filtering for the unique category occurences might help, but then to pass it into a NumPy tensor it won't work, even if it just has 8 different categories in a list.
* I might need to use the very own dataframe in the PyTorch dataloader, extract the encodings to create the embedding columns and only then convert it to a PyTorch tensor.

## Create a dataframe dataloader